In [41]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd


In [42]:
import pickle
import numpy as np

# Load BERT embeddings
with open('bert_embeddings.pkl', 'rb') as f:
    bert_embeddings = pickle.load(f)



In [43]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load BERT embeddings
with open("bert_embeddings.pkl", "rb") as f:
    bert_embeddings = pickle.load(f)

# Load sentiment labels
labels = pd.read_csv(r"C:\Users\pbsns\OneDrive\Documents\MAJOR\xlm-t-main\notebooks\train_labels_combined.txt", header=None).squeeze().tolist()

# Convert labels to NumPy array
labels = np.array(labels)

# Ensure embeddings and labels match
assert len(bert_embeddings) == len(labels), "Mismatch between embeddings and labels length."

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(bert_embeddings, labels, test_size=0.2, random_state=42)

# Reshape data for LSTM
X_train = np.array(X_train)
X_test = np.array(X_test)

print("✅ Data loaded and ready for LSTM training!")


✅ Data loaded and ready for LSTM training!


In [44]:
print(f"X_train shape: {X_train.shape}")

X_train shape: (23532, 3)


In [47]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))


In [48]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [49]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(3, activation="softmax"))  # Assuming 3 sentiment classes


c:\Users\pbsns\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [50]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])



In [51]:
print(X_train.dtype)
print(y_train.dtype)


object
int32


In [52]:
print(X_train[:5])


[[['nicht mehr ganz frisch: George Clooney\'s "Monuments Men" (2013) First Look! ENTV: http via @user\n'
   1
   'nicht mehr ganz frisch george clooneys monuments men 2013 first look entv http via']]

 [['RT @user: „Ich beuge mich meinem Schicksal..“ „Du stehst auf.“ „..ist nicht leicht, sich damit abzufinden..“ „Du stehst nur auf!“ „AM…\n'
   0 'rt beuge schicksal stehst leicht abzufinden stehst']]

 [['roz rulane ki kasam khayi han is confession ne ..  P \n' 2
   'roz rulane ki kasam khayi han confession ne p']]

 [['23e édition des Entretiens de Sologne : le programme du Colloque http Un hommage à l’#arbre, pilier de l’#écosystème\n'
   1
   '23e dition entretiens sologne programme colloque http hommage larbre pilier lcosystme']]

 [['Amaia Montero en el estudio de grabación en Londres preparando una sorpresa muy especial. ¿Qué será?\n'
   1
   'amaia montero estudio grabacin londres preparando sorpresa especial qu ser']]]


In [53]:
print(f"X_train shape: {X_train.shape}")


X_train shape: (23532, 1, 3)


In [62]:
def convert_to_float(val):
    try:
        return float(val)
    except ValueError:
        return np.nan  # or any other placeholder for invalid entries

vectorized_convert = np.vectorize(convert_to_float)
X_train = vectorized_convert(X_train)
X_test = vectorized_convert(X_test)



In [63]:
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)


In [64]:
print(f"X_train dtype after conversion: {X_train.dtype}")


X_train dtype after conversion: float32


In [65]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)


In [66]:
print(np.unique(y_train))
print(np.unique(y_test))


[0 1 2 3]
[0 1 2]


In [67]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
 72/736 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3510 - loss: nan

InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\pbsns\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\pbsns\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "c:\Users\pbsns\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "c:\Users\pbsns\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\pbsns\anaconda3\Lib\asyncio\windows_events.py", line 321, in run_forever

  File "c:\Users\pbsns\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Users\pbsns\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  

In [60]:

# Save model
model.save("lstm_sentiment_model.h5")

print("✅ LSTM model trained and saved!")

✅ LSTM model trained and saved!


In [61]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


ValueError: Invalid dtype: object